# Training of the model on all the datasets

In [ ]:
#we first connect to my google drive in order to collect all the data

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%tensorflow_version 1.x
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K
from keras.utils import to_categorical

In [ ]:
#we load all the datasets 
data_lap = pd.read_csv("/content/drive/My Drive/Tesi_ABSA/dataset/laptops_2014.csv", encoding="latin1", sep="\t")
data_lap = data_lap.fillna(method="ffill")

data_comp = pd.read_csv("/content/drive/My Drive/Tesi_ABSA/dataset/Computer.csv", encoding="latin1", sep="\t")
data_comp = data_comp.fillna(method="ffill")

data_router = pd.read_csv("/content/drive/My Drive/Tesi_ABSA/dataset/Router.csv", encoding="latin1", sep="\t")
data_router = data_router.fillna(method="ffill")

data_speaker = pd.read_csv("/content/drive/My Drive/Tesi_ABSA/dataset/Speaker.csv", encoding="latin1", sep="\t")
data_speaker = data_speaker.fillna(method="ffill")

data_rest = pd.read_csv("/content/drive/My Drive/Tesi_ABSA/dataset/restaurants1.csv", encoding="latin1", sep="\t")
data_rest = data_rest.fillna(method="ffill")

data_hotels = pd.read_csv("/content/drive/My Drive/Tesi_ABSA/dataset/hotels.csv", encoding="latin1", sep="\t")
data_hotels = data_hotels.fillna(method="ffill")

In [ ]:
#creation of a set containing the words

words_lap = set(list(data_lap['TOKEN'].values))
words_comp = set(list(data_comp['TOKEN'].values))
words_router = set(list(data_router['TOKEN'].values))
words_speaker = set(list(data_speaker['TOKEN'].values))
words_rest = set(list(data_rest['TOKEN'].values))
words_hotels = set(list(data_hotels['TOKEN'].values))
words = set({})
words = words.union(words_lap,words_comp,words_router,words_speaker,words_rest,words_hotels)
words.add('PADword')
n_words = len(words)
print(n_words)

In [ ]:
#we create the set containing the unique tags
tags = list(set(data_lap["TAG"].values))
n_tags = len(tags)
print(n_tags)

In [ ]:
#class for get sentences in the format of tuples [(TOKEN,TAG),(TOKEN,TAG)...]
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["TOKEN"].values.tolist(),s["TAG"].values.tolist())]
        self.grouped = self.data.groupby("SENTENCE").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
#we get all the sentences and we print the longest sentence
getter_lap = SentenceGetter(data_lap)
sentences_lap = getter_lap.sentences
print(len(sentences_lap))

getter_comp = SentenceGetter(data_comp)
sentences_comp = getter_comp.sentences
print(len(sentences_comp))

getter_router = SentenceGetter(data_router)
sentences_router = getter_router.sentences
print(len(sentences_router))

getter_speaker = SentenceGetter(data_speaker)
sentences_speaker = getter_speaker.sentences
print(len(sentences_speaker))

getter_rest = SentenceGetter(data_rest)
sentences_rest = getter_rest.sentences
print(len(sentences_rest))

getter_hotels = SentenceGetter(data_hotels)
sentences_hotels = getter_hotels.sentences
print(len(sentences_hotels))

In [ ]:
#we print the first sentence in the above format
print(sentences_router[0])

In [ ]:
#we create the X dataset containing only the padded sentences' tokens
max_len = 100
X = [[w[0]for w in s] for s in sentences_lap]
new_X_lap = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X_lap.append(new_seq)
print(new_X_lap[15])

X = [[w[0]for w in s] for s in sentences_comp]
new_X_comp = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X_comp.append(new_seq)
print(new_X_comp[15])

X = [[w[0]for w in s] for s in sentences_router]
new_X_router = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X_router.append(new_seq)
print(new_X_router[15])

X = [[w[0]for w in s] for s in sentences_speaker]
new_X_speaker = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X_speaker.append(new_seq)
print(new_X_speaker[15])

X = [[w[0]for w in s] for s in sentences_rest]
new_X_rest = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X_rest.append(new_seq)
print(new_X_rest[15])

X = [[w[0]for w in s] for s in sentences_hotels]
new_X_hotels = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X_hotels.append(new_seq)
print(new_X_hotels[15])

In [ ]:
#we also create the padded y 
from keras.preprocessing.sequence import pad_sequences
tags2index = {"O":0,"B-aspect":1,"I-aspect":2}
y_lap = [[tags2index[w[1]] for w in s] for s in sentences_lap]
y_lap = pad_sequences(maxlen=max_len, sequences=y_lap, padding="post", value=tags2index["O"])

y_comp = [[tags2index[w[1]] for w in s] for s in sentences_comp]
y_comp = pad_sequences(maxlen=max_len, sequences=y_comp, padding="post", value=tags2index["O"])

y_router = [[tags2index[w[1]] for w in s] for s in sentences_router]
y_router = pad_sequences(maxlen=max_len, sequences=y_router, padding="post", value=tags2index["O"])

y_speaker = [[tags2index[w[1]] for w in s] for s in sentences_speaker]
y_speaker = pad_sequences(maxlen=max_len, sequences=y_speaker, padding="post", value=tags2index["O"])

y_rest = [[tags2index[w[1]] for w in s] for s in sentences_rest]
y_rest = pad_sequences(maxlen=max_len, sequences=y_rest, padding="post", value=tags2index["O"])

y_hotels = [[tags2index[w[1]] for w in s] for s in sentences_hotels]
y_hotels = pad_sequences(maxlen=max_len, sequences=y_hotels, padding="post", value=tags2index["O"])

In [ ]:
X_tr_lap, X_te_lap, y_tr_lap, y_te_lap = train_test_split(new_X_lap, y_lap, test_size=0.2, random_state=2018)
X_tr_comp, X_te_comp, y_tr_comp, y_te_comp = train_test_split(new_X_comp, y_comp, test_size=0.2, random_state=2018)
X_tr_router, X_te_router, y_tr_router, y_te_router = train_test_split(new_X_router, y_router, test_size=0.2, random_state=2018)
X_tr_speaker, X_te_speaker, y_tr_speaker, y_te_speaker = train_test_split(new_X_speaker, y_speaker, test_size=0.2, random_state=2018)
X_tr_rest, X_te_rest, y_tr_rest, y_te_rest = train_test_split(new_X_rest, y_rest, test_size=0.2, random_state=2018)
X_tr_hotels, X_te_hotels, y_tr_hotels, y_te_hotels = train_test_split(new_X_hotels, y_hotels, test_size=0.2, random_state=2018)
X_tr = X_tr_hotels + X_tr_speaker + X_tr_comp + X_tr_lap + X_tr_router
X_val = X_te_hotels + X_te_speaker + X_te_comp + X_te_lap + X_te_router
y_tr = np.concatenate((y_tr_hotels, y_tr_speaker, y_tr_comp, y_tr_lap, y_tr_router), axis=0)
y_val = np.concatenate((y_te_hotels, y_te_speaker, y_te_comp, y_te_lap, y_te_router), axis=0)

In [ ]:
sess = tf.Session()
K.set_session(sess)
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [ ]:
#we define the function that will be used in the embedding layer of the network
batch_size = 32
def ElmoEmbedding(x):
    return elmo_model(inputs={"tokens": tf.squeeze(tf.cast(x,    tf.string)),"sequence_len": tf.constant(batch_size*[max_len])
                     },
                      signature="tokens",
                      as_dict=True)["elmo"]

In [ ]:
from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda
input_text = Input(shape=(max_len,), dtype=tf.string)
embedding = Lambda(ElmoEmbedding, output_shape=(max_len, 1024))(input_text)
x = Bidirectional(LSTM(units=512, return_sequences=True,
                       recurrent_dropout=0.2, dropout=0.2))(embedding)
x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           recurrent_dropout=0.2, dropout=0.2))(x)
x = add([x, x_rnn])  # residual connection to the first biLSTM
out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)
model = Model(input_text, out)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
#we train the model 
X_tr, X_val = X_tr[:(int(len(X_tr)/32))*32], X_val[:(int(len(X_val)/32))*32]
y_tr, y_val = y_tr[:(int(len(y_tr)/32))*32], y_val[:(int(len(y_val)/32))*32]
y_tr = y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)
y_val = y_val.reshape(y_val.shape[0], y_val.shape[1], 1)

history = model.fit(np.array(X_tr), y_tr, validation_data=(np.array(X_val), y_val),batch_size=batch_size, epochs=10, verbose=1)

# ABSA on Yelp Dataset

In [ ]:
!pip install twython

In [ ]:
#model.save_weights("/content/drive/My Drive/Tesi_ABSA/dataset/Aspect Extraction Task/model_weights.weights")
model.load_weights('/content/drive/My Drive/Tesi_ABSA/dataset/Aspect Extraction Task/model_weights.weights')

In [ ]:
import json
import re
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
#We first need to divide reviews into atomic sentences
#Then we also extract the sentiment of the sentence in order to assign it to all the aspects in the sentence

f_rev = open("/content/drive/My Drive/Tesi_ABSA/dataset/Aspect Extraction Task/yelp_training_set_review.json","r")
f_rev_output = open("/content/drive/My Drive/Tesi_ABSA/dataset/Aspect Extraction Task/review_divided.txt","w+")
count = 0
sid = SentimentIntensityAnalyzer()

#We iterate over all the rows in the yelp dataset (each line is a json)
for line in f_rev:
  #we read the text of the review
  rev_text = json.loads(line)["text"]
  #we read the user_id
  rev_user = json.loads(line)["user_id"]
  #we read the business_id
  rev_business = json.loads(line)["business_id"]
  #we tokenize the sentence using the sent_tokenize of nltk
  rev_splitted = sent_tokenize(rev_text)

  #splitted sentences
  res = []
  #sentiment scores
  compounds = []
  neg = []
  neu = []
  pos = []
  #We iterate over the sentences in the current review and predict the sentiment scores
  for i in range (len(rev_splitted)):
    ss = sid.polarity_scores(rev_splitted[i])
    res.append(rev_splitted[i])
    compounds.append(ss['compound'])
    neg.append(ss['neg'])
    neu.append(ss['neu'])
    pos.append(ss['pos'])
  sentence_id = 0
  #We write all the extracted values on the f_rev_output file (each value is separated by \t)
  for sentence,comp,nega,neut,posi in zip(res,compounds,neg,neu,pos):
    f_rev_output.write(str(count)+"\t")
    f_rev_output.write(str(sentence_id)+"\t")
    f_rev_output.write(str(comp)+"\t")
    f_rev_output.write(str(nega)+"\t")
    f_rev_output.write(str(neut)+"\t")
    f_rev_output.write(str(posi)+"\t")
    f_rev_output.write(rev_user+"\t")
    f_rev_output.write(rev_business+"\t")
    splitted_sentence = re.findall(r"[\w']+|[.,!?;]", sentence)
    for token in splitted_sentence:
      f_rev_output.write(token+" ")
    f_rev_output.write("\n")
    sentence_id += 1
    
  count += 1
f_rev_output.close()